In [68]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows",10) 

In [69]:
path='../Data'

In [70]:
"""
讀取df : covid stat data
"""
df = pd.read_csv(f"{path}/owid-covid-data.csv")
df['date'] = pd.to_datetime(df['date'])

1. 變種病毒欄位合併

In [71]:
"""
讀取sars : 變種病毒統計資料
"""
sars = pd.read_csv(f"{path}/SARS-CoV-2.csv")

"""
拆解年份與第幾週
"""
def year_week(d):
    d = d.split('-')
    y, w = d[0], d[1]
    return y, w.zfill(2)

sars['year'], sars['week'] = zip(*sars['year_week'].apply(year_week))

In [72]:
"""
保留需要用到的欄位，並將`country`改名為`location`，以便後續與OWID-COVID-DATA進行合併
"""
sars = sars[['country', 'variant', 'number_detections_variant', 'year', 'week']]
sars.rename(columns={'country':'location'}, inplace=True)

"""
刪除重複資料
"""
sars.sort_values(['number_detections_variant'], ascending=False, inplace=True)
sars.drop_duplicates(keep='first', inplace=True)

"""
計算各國家各週各變種病毒的案例，並且只取每週的加總值
"""
sars1 = sars.groupby(['location', 'year', 'week']).sum().reset_index()
sars1 = sars1.fillna(0)
sum_of_variant = sars1['number_detections_variant']

"""
計算各種病毒的數量
"""
sars1 = sars.groupby(['location', 'year', 'week', 'variant'])['number_detections_variant'].sum().unstack().reset_index()
sars1 = sars1.fillna(0)
sars1['number_detections_variant'] = sum_of_variant
"""
幫OWID-COVID-DATA新增`year`與`week`共兩個變數，以利合併(left join)
"""
df['year'] = df['date'].dt.year
df['week'] = df['date'].dt.week
df['year'] = df['year'].astype(str)
df['week'] = df['week'].apply(lambda x:str(x).zfill(2))

In [73]:
"""
合併df, sars1兩個欄位
"""
df1 = pd.merge(df, sars1, on=['location', 'year', 'week'], how='left')

"""
由於變種病毒僅記錄30個國家的資訊，因此其餘國家皆補為0
"""
df1 = df1.fillna(0)

"""
將`year`與`week`兩個欄位刪除
"""
df1.drop(['year', 'week'], 1, inplace=True)

2. 實施的防疫政策合併

In [74]:
"""
rep : 防疫政策數據
"""
rep = pd.read_csv(f"{path}/response_graphs_data_2021-12-09.csv")

"""
將`Country`改名為`location`，以便後續合併
"""
rep.rename(columns={'Country':'location'}, inplace=True)

In [75]:
"""
確認有國家一天頒布兩個以上的政策，且並無存在政策只出現過一次的，因此以OneHotEnocoding的方式進行資料合併 (此資料為建模用)
首先創建一個表包含2020-01-01至2020-12-10所有國家的空表格
"""
temp = [d.strftime('%Y-%m-%d') for d in pd.date_range('2020-01-01', '2021-12-10')]
data = []
for x in list(set(rep['location'])):
    for d in temp:
        data.append((x, d))
new_rep = pd.DataFrame(data, columns=['location', 'date'])

In [76]:
"""
為每一種政策新增一個空的欄位，從開始政策到結束政策都為1，沒有停止時間則後續全部都是1
"""

for x in list(set(rep['Response_measure'])):
    new_rep[x] = np.nan

for i in range(len(rep)):
    l, r, s, e = rep.iloc[i, 0], rep.iloc[i, 1], rep.iloc[i, 2], rep.iloc[i, 3]
    if not pd.isna(e):
        new_rep[r][(new_rep['location']==l) & (new_rep['date']>=s) & (new_rep['date']<=e)] = 1
    else:
        new_rep[r][(new_rep['location'] == l) & (new_rep['date']>=s)] = 1

new_rep = new_rep.fillna(0)
new_rep['SumOfResponseMeasure'] = new_rep.iloc[:, 2:].sum(1)

In [77]:
"""
合併new_rep與df1資料表
"""
new_rep['date'] = pd.to_datetime(new_rep['date'])
df2 = pd.merge(df1, new_rep, on=['date', 'location'], how='left')
df2 = df2.fillna(0)

In [79]:
"""
刪減df2資料，只留需要的欄位、改成適合閱讀的欄位名稱，並儲存
"""
df2 = df2[['location', 'date', 'new_cases', 'new_deaths', 'icu_patients', 'SumOfResponseMeasure', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred']+list(sars1)[3:]]
column_dict = {'location':'Location', 
                 'date':'Date',
                 'new_cases': 'New Cases',
                 'new_deaths': 'New Deaths',
                 'icu_patients': 'ICU Patients',
                 'SumOfResponseMeasure': 'Prevention Policy',
                 'people_vaccinated_per_hundred': 'People Vaccinated Per Hundred',
                 'people_fully_vaccinated_per_hundred' : 'People Full Vaccinated Per Hundred',
                 'number_detections_variant': 'Number of Detections Variant',
                }
column_dict.update({i:i for i in list(sars1)[3:-1]})
df2.columns = df2.columns.map(column_dict)
df2.to_csv(f'{path}/corr_and_regression_data.csv', index=False, encoding='utf-8-sig')